Here we will demonstrate the utility of quantum information through Superdense Coding. Superdense coding is a quantum communication protocol to transmit two classical bits of information from a sender (who will be named Alice) to a receiver (who will be named Bob). The novelty of superdense coding is that Alice will only need to send a single qubit!! The catch is that Alice and Bob will need to pre-share an entangled state. We will create a third party, named Charlie, who is responsible for the creation and sharing of the entangled state.

I am attepting to follow the conventions of Nielsen and Chuang on page 97. You can find free info at https://en.wikipedia.org/wiki/Superdense_coding

I will assume that the reader is familiar with the Pauli matrices, the Hadamard gate, and the CNOT gate.

In [1]:
import numpy as np

# First we should define the gates we will need as numpy arrays. I have chosen the computational basis.

# Single qubit gates
X = np.array([[0,1],[1,0]])
Y = np.array([[0,-1j],[1j,0]])
Z = np.array([[1,0],[0,-1]])
I = np.eye(2)
H = np.array([[1,1],[1,-1]])/np.sqrt(2)

# Double qubit gates
CNOT = np.array([
    [1,0,0,0],
    [0,0,0,1],
    [0,0,1,0],
    [0,1,0,0]]) # Note that this is the matrix representation when the right most qubit is the control.

$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$$
$$\newcommand{\bra}[1]{\left\langle{#1}\right|}$$

We will use the common convention that 
$\ket{0} = \begin{bmatrix}
1 \\
0 \end{bmatrix}$
and
$\ket{1} = \begin{bmatrix}
0 \\
1 \end{bmatrix}$

In [2]:
# Define our single qubit states
bra0 = np.array([1,0])
bra1 = np.array([0,1])

Now we turn it over to Charlie to prepare our entangled state.

In [3]:
# Charlie starts with both qubits in the state |00>. 
# The left qubit is intended for Bob and the right qubit is intended for Alice, i.e. |Bob,Alice>
psi = np.kron(bra0,bra0)

# Now Charlie can create a Bell State, which is an entangled state
# Charlie applies a Hadamard gate to the rightmost qubit (control)
psi = np.kron(I,H)@psi

# To finish entangling the states, Charlie applies a CNOT gate
psi = CNOT @ psi

# Now Charlie gives the left qubit to Bob and the right qubit to Alice



In [4]:
# Alice encodes her message onto the qubit

intended_msg = '11'

def encode_msg_on_qubit(intended_msg,psi):
    # intended_msg should be '00','01','10','11'
    # Note that we have an identity matrix acting on Bob's qubit
    # so Alice doesn't need to be able to be in possession of it.
    # Alice only applies the gate to her qubit (the right one)
    if intended_msg == '00':
        np.kron(I,I) @ psi # Which is to say, do nothing
    elif intended_msg == '01':
        psi = np.kron(I,Z) @ psi
    elif intended_msg == '10':
        psi = np.kron(I,X) @ psi
    elif intended_msg == '11':
        psi = np.kron(I,1j*Y) @ psi
    else:
        print('Not a sendable message, sending 00')
    return psi

psi = encode_msg_on_qubit(intended_msg,psi)

# Then Alice sends her single qubit to Bob

In [7]:
# Bob decodes the message
def decode_msg(psi):
    psi = CNOT @ psi
    psi = np.kron(I,H) @ psi
    measured_psi = psi.conj().T*psi
    # Since that will always be real valued we can do
    measured_psi = abs(measured_psi)
    msg = int(np.argmax(measured_psi))
    return format(msg,'02b')

print('Alice encoded and sent:   ',intended_msg)
print('Bob received and decoded: ',decode_msg(psi))
print('The messages are equal?   ', intended_msg == str(decode_msg(psi)))

Alice encoded and sent:    11
Bob received and decoded:  11
The messages are equal?    True
